In [ ]:
class PSURTemplate:
    def __init__(self, logo_path, drug_name, anda_number, reporting_period):
        self.logo_path = '/Users/rahulgupta/Desktop/jub.png'
        self.drug_name = drug_name
        self.anda_number = anda_number
        self.reporting_period = reporting_period
        self.doc = Document()
        self._setup_document()
    
    def _setup_document(self):
        """Set up document with margins and headers on every page"""
        for section in self.doc.sections:
            # Set margins
            section.top_margin = Inches(1.5)  # Increased for header space
            section.bottom_margin = Inches(0.5)
            section.left_margin = Inches(0.75)
            section.right_margin = Inches(0.75)
            section.header_distance = Inches(0.5)
            
            # Add header to section
            self._setup_header(section)
    
    def _setup_header(self, section):
        """Set up header for a section"""
        header = section.header
        
        # Clear existing header content
        for paragraph in header.paragraphs:
            p = paragraph._element
            p.getparent().remove(p)
        
        # Add header table

        total_width = Inches(7.0)
        header_table = header.add_table(rows=1, cols=2, width=total_width)
        header_table.autofit = False
        header_table.columns[0].width = Inches(2.0)
        header_table.columns[1].width = Inches(5.0)
        
        # Left cell - Logo
        left_cell = header_table.cell(0, 0)
        left_cell.vertical_alignment = WD_CELL_VERTICAL_ALIGNMENT.TOP
        left_para = left_cell.paragraphs[0]
        left_para.alignment = WD_ALIGN_PARAGRAPH.LEFT
        
        if os.path.exists(self.logo_path):
            try:
                run = left_para.add_run()
                run.add_picture(self.logo_path, width=Inches(1.125))
            except Exception as e:
                print(f"Warning: Could not add logo image: {e}")
                self._add_text_logo(left_para)
        else:
            print(f"Warning: Logo file not found at {self.logo_path}")
            self._add_text_logo(left_para)
        
        left_para.paragraph_format.space_after = Pt(0)
        
        # Right cell - Drug info
        right_cell = header_table.cell(0, 1)
        right_cell.vertical_alignment = WD_CELL_VERTICAL_ALIGNMENT.TOP
        right_para = right_cell.paragraphs[0]
        right_para.alignment = WD_ALIGN_PARAGRAPH.RIGHT
        right_run = right_para.add_run(
            f"{self.drug_name}\n"
            f"(A)NDA: {self.anda_number}\n"
            f"Reporting Period: {self.reporting_period}"
        )
        right_run.font.name = 'Arial'
        right_run.font.size = Pt(9)
        right_para.paragraph_format.line_spacing = Pt(11)
        right_para.paragraph_format.space_after = Pt(0)
        
        # Remove table borders
        self._remove_table_borders(header_table)
    
    def _add_text_logo(self, paragraph):
        """Add text-based logo as fallback"""
        run = paragraph.add_run("JUBILANT\nGENERICS")
        run.font.name = 'Arial'
        run.font.bold = True
        run.font.size = Pt(10)
    
    def _remove_table_borders(self, table):
        """Remove all borders from a table"""
        tbl = table._tbl
        tblPr = tbl.tblPr
        if tblPr is None:
            tblPr = OxmlElement('w:tblPr')
            tbl.insert(0, tblPr)
        
        # Remove existing borders
        for child in tblPr:
            if child.tag.endswith('tblBorders'):
                tblPr.remove(child)
        
        tblBorders = OxmlElement('w:tblBorders')
        for bdtype in ['top', 'left', 'bottom', 'right', 'insideH', 'insideV']:
            el = OxmlElement(f'w:{bdtype}')
            el.set(qn('w:val'), 'none')
            el.set(qn('w:sz'), '0')
            el.set(qn('w:space'), '0')
            el.set(qn('w:color'), 'auto')
            tblBorders.append(el)
        tblPr.append(tblBorders)
    
    def add_cover_page(self, drug_name, anda_number, reporting_period, tablet_name):
        """Add cover page"""
        # Main title
        title_para = self.doc.add_paragraph()
        title_para.paragraph_format.space_before = Pt(24)
        title_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
        title_run = title_para.add_run("PERIODIC ADVERSE DRUG EXPERIENCE REPORT (PSUR)")
        title_run.font.name = 'Arial'
        title_run.font.size = Pt(16)
        title_run.font.bold = True
        title_para.paragraph_format.space_after = Pt(20)
        
        # FOR section
        for_para = self.doc.add_paragraph()
        for_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
        for_run = for_para.add_run("FOR")
        for_run.font.name = 'Arial'
        for_run.font.size = Pt(16)
        for_run.font.bold = True
        for_para.paragraph_format.space_after = Pt(16)
        self.doc.add_paragraph()
        
        # Tablet name
        tablet_para = self.doc.add_paragraph()
        tablet_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
        tablet_run = tablet_para.add_run(drug_name)
        tablet_run.font.name = 'Arial'
        tablet_run.font.size = Pt(14)
        tablet_run.font.bold = True
        tablet_para.paragraph_format.space_after = Pt(16)
        
        # Marketing Authorization Holder
        mah_para = self.doc.add_paragraph()
        mah_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
        mah_run = mah_para.add_run("Marketing Authorization Holder:")
        mah_run.font.name = 'Arial'
        mah_run.font.size = Pt(14)
        mah_run.font.bold = True
        mah_para.paragraph_format.space_after = Pt(2)
        
        company_para = self.doc.add_paragraph()
        company_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
        company_run = company_para.add_run("Jubilant Generics Limited 1-A, Sector 16-A, Noida- 201 301, UP, India`")
        company_run.font.name = 'Arial'
        company_run.font.size = Pt(14)
        company_run.font.bold = True
        company_para.paragraph_format.space_after = Pt(32)
        
        # Products section
        products_para = self.doc.add_paragraph()
        products_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
        products_text_run = products_para.add_run(f"Products: {tablet_name}\n")
        products_text_run.font.name = 'Arial'
        products_text_run.font.size = Pt(14)
        products_text_run.font.bold = True
        products_para.paragraph_format.space_after = Pt(32)
        
        # Period covered
        period_para = self.doc.add_paragraph()
        period_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
        period_run = period_para.add_run(f"Period covered by this report: {reporting_period}")
        period_run.font.name = 'Arial'
        period_run.font.size = Pt(14)
        
        # Footer disclaimer
        disclaimer_text = ("This periodic adverse drug experience report (PADER) contains confidential and proprietary "
                          "information of Jubilant Generics Limited, which is in part, the unpublished research findings of its "
                          "scientists. This report should not be copied, distributed, or published, in whole or in part, or be "
                          "referred to in any publication without prior written authorization from Jubilant Generics Limited.")
        
        disclaimer_para = self.doc.add_paragraph()
        disclaimer_para.paragraph_format.space_before = Pt(200)
        disclaimer_para.alignment = WD_ALIGN_PARAGRAPH.LEFT
        disclaimer_run = disclaimer_para.add_run(disclaimer_text)
        disclaimer_run.font.name = 'Arial'
        disclaimer_run.font.size = Pt(8)
    
    def add_index_page(self):
        """Add index page"""
        self.doc.add_page_break()
        
        # Index content
        index_para = self.doc.add_paragraph()
        index_para.alignment = WD_ALIGN_PARAGRAPH.LEFT
        index_run = index_para.add_run("INDEX")
        index_run.font.name = 'Arial'
        index_run.font.size = Pt(12)
        index_run.font.bold = True
        index_run.underline = True
        index_para.paragraph_format.space_after = Pt(12)
        
        items = [
            "LIST OF SERIAL NUMBER OF PREVIOUS REPORTS",
            "EXECUTIVE SUMMARY",
            "LIST OF ABBREVIATION",

            "1 INTRODUCTION",
            "2 WORLD WIDE MARKETING AUTHORISATION STATUS",
            "3 ACTION TAKING IN THE REPORTINIG INTERVAL FOR SAFETY REASONS",
            "4 CHANGES TO REFERENCE AND SAFETY INFORMATION",
            '5 ESTIMATED EXPOSURE AND USE PATTERNS',
            "   5.1 GENERAL CONSIDERATIONS",
            "   5.2 CUMULATIVE SUBJECT EXPOSURE IN CLINICAL TRAILS",
            "   5.3 CUMULATIVE AND INTERVAL PATIENT EXPOSURE FROM MARKETING EXPERIENCE",
            '',
            "6.  DATA IN SUMMARY TABULATIONS",
            "   6.1 REFERENCE INFORMATION",
            "   6.2 CUMULATIVE SUMMARY TABULATION OF SERIOUS ADVERSE EVENTS FROM CLINICAL  TRAIL",
            "   6.3 CUMULATIVE AND INTERVAL SUMMARY TABULATION FROM POST-MARKETING DATA SOURCES",
                                    ,
            "7  SUMMARIES OF SIGINFICANT FINDING FROM CLINICAL TRAIL DURING THE REPORTING INTERVAL",
            "   7.1 COMPLETED CLINICAL TRAIL",
            "   7.2 ONGOING CLINICAL TRIAL",
            "   7.3 LONG TERM FOLLOW UP",
            "   7.4 OTHER THERAPEUTIC USE OF MEDICAL PRODUCT",
            "   7.5 NEW SAFETY DATA RELATED TO FIXED COMBINATION THERAPIES",
            "8 FINDINGS FROM NON-INTERVENTIONAL STUDIES",
            "9 INFORMATION FROM OTHER CLINICAL TRAILS AND SOURCES",
            "   9.1 OTHER CLINICAL TRAILS",
            "   9.2 MEDICATION ERRORS",
            "10 NON-CLINICAL DATA",
            "11 LITERATURE",
            
            "   11.1 LITERATURE ARTICLES ON PREGNANCY OUTCOME (INCLUDING TERMUNATION) WITH/WITHOUT ADVERSE OUTCOMES",
            "   I.   PUBLISHED LITERATURE ARTICLE HAVING SPECIFIC SAFETY IMPLICATION/SAFETY ISSUE WITH LEVETIRACETAM USE IN PREGNANCY",
            "   II.  PUBLISHED LITERATURE ARTICLE HAVING NO SPECIFIC SAFETY IMPLICATION/SAFETY ISSUE WITH LEVETIRACETAM USE IN PREGNANCY",
            
            "   11.2 LITERATURE ARTICLES WITH USE IN PAEDIATRIC POPULATIONS ",
            "   11.3 LITERATURE ARTICLES WITH COMPASSIONATE SUPPLY, NAMED PATIENT USE ",
            "   11.4 LITERATURE ARTICLES ON LACK OF EFFICACY ",
            "   11.5 LITERATURE ARTICLES ON ASYMPTOMATIC OVERDOSE , ABUSE OR MISUSE",
            "   11.6 LITERATURE ARTICLES WITH MEDICATION ERROR WHERE NO ADVERSE EVENT OCCURRED ",
            "   11.7 LITERATURE ARTICLES WITH IMPORTANT NON-CLINICAL SAFETY RESULTS",

            "12 OTHER PERIODIC REPORTS",
            "13 LACK OF EFFICACY IN CONTROLLED CLINICAL TRAILS",
            "14 LATE BREAKING INFORMAATION ",
            "15 OVERVIEW OF SIGNALS: NEW, ONGOING , OR CLOSED",
            "   15.1 BREIF DESCRIPTION OF SIGNAL DETECTION METHOD",
            "   15.2 SIGNAL TABULATIONS",
            "16 SIGNAL AND RISK EVALUATION ",
            "   16.1 SUMMARY OF SAFETY CONCERNS",
            "   16.2 SIGNAL EVALUATION",
            "   16.3 EVALUATIOIN OF RISK AND NEW INFORMATION",
            "   16.4 EFFECTIVENESS OF RISK MINIMISATION( IF APPLICABLE)",
            "17 BENIFIT EVALUATION",
            "   17.1 IMPORTANT BASELINE EFFICACY ANF EFFECTIVENESS INFORMATION",
            "   17.2 NEWLY IDENTIFICATION INFORMATON ON EFFICACY AND EFFECTIVENESS",
            "   17.3 CHARACTERISATION OF BENEFITS"
            "18 INTEGRATED BENEFIT-RISK ANALYSIS FOR AUTHORISED INDICATIONS",
            "   18.1 BENEFIT-RISK CONTEXT-MEDICAL NEED AND IMPORTANT ALTERNATIVES",
            "   18.2 BENIFIT-RISK ANALYSIS EVALUATION",
            "19 CONCLUSION AND ACTIONS",
            "20 APPENDICES",
            "   20.1 REFERENCE INFORMATION ",
            "   20.2 CURRENT MARKETING AUTHORIZATION STATUS",
            "   20.3 CUMULATIVE SUMMARY TABULATION OF SAEs FROM CLINICAL TRAILS AND CUMULATIVE AND INTERVAL SUMMARY TABULATIONS OF SERIOUS NON-SERIOUS ADVERSE REACTION FROM POST-MARKETINF DATA SOURCES",
            "   20.4 TABULAR SUMMARY OF SAFETY SIGNALS."
            "   20.5 LISTING OF ALL MARKETING AUTHORICATION HOLDER-SPONSORED NON-INTERVENTIONAL STUDIES CONDUCTED WITH PRIMARY AIM OF IDETIFCATING, CHARATERSING OR QUANTIFYING A SAFETY HAZARD, CONFIRMING THE SAFETY PROFILE OF THE MEDICINAL PRODUCT , OR OF MEADURING THE EFFECTIVENESS OF RISK MANAGEMENT MEASURES.",
            "   20.6 LIST OF THE SOURCES OF INFORMAION USEED TO PREPARE THE ADDENDIUM"
            '',
        ]
        for text in items:
            if not text.strip():
                self.doc.add_paragraph()
                continue
            p = self.doc.add_paragraph(text)
            p.alignment = WD_ALIGN_PARAGRAPH.LEFT
            r = p.runs[0]
            r.font.name = 'Arial'
            r.font.size = Pt(11)
            if text.split()[0].rstrip('.').isdigit():
                r.font.bold = True
            p.paragraph_format.space_after = Pt(6)

Levetiracetam Periodic Safety Update Report
Jubilant Generics Limited
30-Nov-2021 to 30-Nov-2024
LIST OF SERIAL NUMBERS OF PREVIOUS REPORTS - Page 4
EXECUTIVE SUMMARY - Page 5
LIST OF ABBREVIATIONS - Page 11
INTRODUCTION - Page 12
WORLD WIDE MARKETING AUTHORISATION STATUS - Page 18
ACTION TAKEN IN THE REPORTING INTERVAL FOR SAFETY REASONS - Page 20
CHANGES TO REFERENCE SAFETY INFORMATION - Page 20
ESTIMATED EXPOSURE AND USE PATTERNS - Page 21
General Considerations (5.1) - Page 21
Cumulative Subject Exposure in Clinical Trials (5.2) - Page 21
Cumulative and Interval Patient Exposure from Marketing Experience (5.3) - Page 36
DATA IN SUMMARY TABULATIONS (6) - Page 37
Reference Information (6.1) - Page 37
Cumulative Summary Tabulations of Serious Adverse Events from Clinical Trials (6.2) - Page 37
Cumulative and Interval Summary Tabulations from Post-Marketing Data Sources (6.3) - Page 37
SUMMARIES OF SIGNIFICANT FINDINGS FROM CLINICAL TRIALS DURING THE REPORTING INTERVAL (7) - Page 39
Co